In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm
# from kw_r2plus1d_video_swin_transformer import SwinTransformer3D
batchsz = 4
frame_nums=15
start_epoch = 0
test_flage=False
model_name ="fullshot and ast"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()
# device = torch.device('cuda:1' if USE_CUDA else 'cpu')

In [2]:
logger.debug('============={}+Video swin transformerTrain Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:1' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2024-03-26 18:54:51,405]::387438924::DEBUG::=============fullshot and ast+Video swin transformerTrain Strat!=============


True
학습을 진행하는 기기: cuda:1


In [3]:
def select_frame_data(datas,frame_num):
    data_set=[]
    begin_num=0
    x=128/frame_num
    indexes=[None] * frame_num
    for i in range(frame_num):
        indexes[i]=int(begin_num)
        begin_num+=x
    for data in datas:
        video_frames = [None] * 2
        video_frames[0]=[]
        video_frames[1]=[]
        for index in indexes:
            video_frames[0].append(data[0][index])
        video_frames[1].append(data[1])
        data_set.append(video_frames)
        del video_frames
        del data 
        gc.collect()
    del datas
    gc.collect()
    return data_set
print(1)

1


In [4]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [5]:
def datasize(type,batchsz):
    if type=='train':
        i=5076#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1701#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

1


In [6]:

save_model_file_path = './save_swintransformer_folder/{}_{}.{}'
mtcnn= MTCNN(image_size=224, margin=0, min_face_size= 20, thresholds=[0.6, 0.7, 0.7], post_process=True)
# test_set_data=getdata("/home/ssrlab/qx/code/test/video-swin-transformer-pytorch/data/15Frames/test_set.dat","test")
# test_set_data=test_set_data[:datasize("test",batchsz)]
print(1)

1


In [7]:
def reshape_to_expected_traininput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i],dataset[4][i],dataset[5][i]))
    result = result[:datasize("train",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []
    x4_list = []
    x5_list = []
    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])
        x4_list.append(result[i][4])
        x5_list.append(result[i][5])
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list), np.stack(x4_list), np.stack(x5_list))
print(1)

1


In [8]:
def reshape_to_expected_validinput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i],dataset[4][i],dataset[5][i]))
    result = result[:datasize("valid",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []
    x4_list = []
    x5_list = []
    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])
        x4_list.append(result[i][4])
        x5_list.append(result[i][5])
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list), np.stack(x4_list), np.stack(x5_list))
print(1)

1


In [9]:
validation_set_data=getdata("/home/ssrlab/qx/code/test/video-swin-transformer-pytorch/data/15Frames/multidata/1700valid_set.dat","valid")
validation_set_data=reshape_to_expected_validinput(validation_set_data)
print(1)

1


In [10]:
print(type(validation_set_data[2][0]))

<class 'numpy.ndarray'>


In [11]:
train_set_data=getdata("/home/ssrlab/qx/code/test/video-swin-transformer-pytorch/data/15Frames/multidata/5076train_set.dat","train")
train_set_data=reshape_to_expected_traininput(train_set_data)
print(1)

1


In [12]:
def imagecrop(imagedata,batchsz):
    # convert_tensor= transforms.ToTensor()
    output=[]
    for j in range(batchsz):#비디오 데이터 batchsz(9) 개 있다
        firstoutput=[]
        for i in range(len(imagedata[j])):#imagedata에 15개의 프레임이 있다
            # temp = convert_tensor(imagedata[i])
            #이미지를 나누
            temp = imagedata[j][i].unfold(1,112,112).unfold(2,112,112)
            temp = rearrange(temp, 'c h1 w1 h w -> (h1 w1) c h w')
            firstoutput.append(temp)
        firstoutput = torch.stack(firstoutput,0)
        output.append(firstoutput)
    output = torch.stack(output,0)
    return output
print(1)

1


In [13]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [14]:
class ChalearnDataset(Dataset):
    def __init__(self,audiodata,facedata,tagdata,transform=None):
        self.audiodata=audiodata
        self.facedata=facedata
        self.tagdata=tagdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.facedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        audiodata = self.audiodata[idx]
        audiodata = torch.FloatTensor(audiodata)
        facedata=self.facedata[idx]
        facedata=torch.FloatTensor(facedata)
        # image_data=image_data.reshape(15,3,224,224)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return audiodata,facedata,big_five_sorces
print(1)

1


In [15]:
num_workerssz = 20
lr = 3e-05
epochs = 120
#device = torch.device('cuda')
# train_set_data = reshape_to_expected_input(train_set_data)
# val_set_data = reshape_to_expected_input(validation_set_data)
print(1)

1


In [16]:
train_set_data = ChalearnDataset(audiodata=train_set_data[0],facedata = train_set_data[2],tagdata=train_set_data[5],transform=transform)
val_set_data = ChalearnDataset(audiodata=validation_set_data[0],facedata = validation_set_data[2], tagdata=validation_set_data[5],transform=transform)
train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [17]:
model=SwinTransformer3D()#num_frames=frame_nums,batchsz = batchsz
model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    # criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

Some weights of AlbertModel were not initialized from the model checkpoint at text_model/ and are newly initialized: ['albert.pooler.weight', 'albert.pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=131


/home/ssrlab/anaconda3/envs/qx/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


1


In [18]:
# from torchinfo import summary

# summary(model, input_size = ((8,15,224,224,3),(8,24,1319,1)), col_names = ['input_size','output_size','num_params'])

In [19]:
def extract_ocean(data,batch_size):
    o=[]
    c=[]
    e=[]
    a=[]
    n=[]
    for i in range(batch_size):
        o.append(data[i][0].item())
        c.append(data[i][1].item())
        e.append(data[i][2].item())
        a.append(data[i][3].item())
        n.append(data[i][4].item())
    o = torch.tensor(o) 
    c = torch.tensor(c) 
    e = torch.tensor(e) 
    a = torch.tensor(a) 
    n = torch.tensor(n) 
    return o,c,e,a,n

In [20]:
with torch.cuda.device(1):
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        train_avg_o_loss = 0
        train_avg_c_loss = 0
        train_avg_e_loss = 0
        train_avg_a_loss = 0
        train_avg_n_loss = 0
        val_avg_o_loss = 0
        val_avg_c_loss = 0
        val_avg_e_loss = 0
        val_avg_a_loss = 0
        val_avg_n_loss = 0
        for audiodata,facedata,big_five_data in tqdm(train_dataloader):
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            audiodata = audiodata.to(device)
            facedata = facedata.to(device)
            big_five_data = big_five_data.to(device)
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(facedata,audiodata)  # 모델의 예측 결과를 저장합니다.
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다. 
            optimizer.step()  
            train_avg_loss += loss  
        train_avg_loss=train_avg_loss/len(train_dataloader)
        with torch.no_grad():#validate
            for audiodata,facedata,big_five_data in tqdm(val_dataloader):
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                audiodata = audiodata.to(device)
                facedata = facedata.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(facedata,audiodata)
                val_loss = criterion(hypothesis, big_five_data)
                hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
                big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
                val_o_loss = criterion(hypothesiso, big_five_datao) 
                val_c_loss = criterion(hypothesisc, big_five_datac) 
                val_e_loss = criterion(hypothesise, big_five_datae) 
                val_a_loss = criterion(hypothesisa, big_five_dataa) 
                val_n_loss = criterion(hypothesisn, big_five_datan)  
                val_avg_loss += val_loss
                val_avg_o_loss += val_o_loss
                val_avg_c_loss += val_c_loss
                val_avg_e_loss += val_e_loss
                val_avg_a_loss += val_a_loss
                val_avg_n_loss += val_n_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)
            val_avg_o_loss=val_avg_o_loss/len(val_dataloader)
            val_avg_c_loss=val_avg_c_loss/len(val_dataloader)
            val_avg_e_loss=val_avg_e_loss/len(val_dataloader)
            val_avg_a_loss=val_avg_a_loss/len(val_dataloader)
            val_avg_n_loss=val_avg_n_loss/len(val_dataloader)
        torch.cuda.empty_cache()
        # if (1-val_avg_loss)>=(1-max_value):
        max_value=val_avg_loss
        start_epoch+=1
        if (i + 1) % 5 == 0:
            torch.save({
                    'epoch': i+1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': val_avg_loss,
                }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , 1-val_avg_o_loss:{:.4f} , 1-val_avg_c_loss:{:.4f} , 1-val_avg_e_loss:{:.4f} , 1-val_avg_a_loss:{:.4f} , 1-val_avg_n_loss:{:.4f}'.format(i+1, 1-val_avg_loss, 1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss))
    

100%|██████████| 425/425 [00:35<00:00, 11.86it/s]
[2024-03-26 19:01:36,308]::2745825432::DEBUG::Epoch: 1 , 1-MAE: 0.8605 , 1-val_avg_o_loss:0.8618 , 1-val_avg_c_loss:0.7874 , 1-val_avg_e_loss:0.9014 , 1-val_avg_a_loss:0.8580 , 1-val_avg_n_loss:0.8938
100%|██████████| 425/425 [00:36<00:00, 11.78it/s]
[2024-03-26 19:07:01,885]::2745825432::DEBUG::Epoch: 2 , 1-MAE: 0.8810 , 1-val_avg_o_loss:0.8918 , 1-val_avg_c_loss:0.8925 , 1-val_avg_e_loss:0.8978 , 1-val_avg_a_loss:0.8902 , 1-val_avg_n_loss:0.8327
100%|██████████| 425/425 [00:35<00:00, 11.82it/s]
[2024-03-26 19:12:27,553]::2745825432::DEBUG::Epoch: 3 , 1-MAE: 0.8861 , 1-val_avg_o_loss:0.8940 , 1-val_avg_c_loss:0.8877 , 1-val_avg_e_loss:0.9039 , 1-val_avg_a_loss:0.8680 , 1-val_avg_n_loss:0.8770
100%|██████████| 425/425 [00:36<00:00, 11.68it/s]
[2024-03-26 19:17:53,898]::2745825432::DEBUG::Epoch: 4 , 1-MAE: 0.8964 , 1-val_avg_o_loss:0.9111 , 1-val_avg_c_loss:0.8874 , 1-val_avg_e_loss:0.8908 , 1-val_avg_a_loss:0.8963 , 1-val_avg_n_loss:0.8

In [21]:
# from torchinfo import summary

# summary(model, input_size = (4,15,4,3,224,224))